In [1]:
import pickle
from collections import Iterable
import pandas as pd
from datetime import datetime

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [2]:
admissions = pd.read_csv('ADMISSIONS.csv')
admissions = pd.DataFrame().assign(ROW_ID=admissions['ROW_ID'], SUBJECT_ID=admissions['SUBJECT_ID'], HADM_ID=admissions['HADM_ID'], ADMITTIME=admissions['ADMITTIME'], DIAGNOSIS=admissions['DIAGNOSIS'])
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')

In [3]:
subject_id_HF = list(admissions[admissions.DIAGNOSIS.str.contains('HEART FAILURE', na=False)]['SUBJECT_ID'])
subject_id_COPD = list(admissions[admissions.DIAGNOSIS.str.contains('COPD', na=False)]['SUBJECT_ID'])
subject_id_KIDNEY = list(admissions[admissions.DIAGNOSIS.str.contains('Kidney', na=False)]['SUBJECT_ID'])

In [143]:
dict_ICD9 = {}
dict_cnt = 0

In [144]:
diags = []
labels = []
times = []
for s_id in subject_id_HF:
  adm = admissions[admissions.SUBJECT_ID == s_id]
  adm.sort_values(by=['ADMITTIME'], inplace = True)
  adm = adm.reset_index()
  if len(adm) <= 2:
    continue
  diag = []
  label = 1
  time = []
  for i in range(len(adm)-1):
    date_obj = datetime.strptime(adm['ADMITTIME'][i][2:], '%y-%m-%d %H:%M:%S')\
                - datetime.strptime(adm['ADMITTIME'][len(adm)-2][2:], '%y-%m-%d %H:%M:%S')
    time.append(0 - date_obj.days)

    dg = diagnoses[diagnoses.HADM_ID == adm['HADM_ID'][i]].reset_index()
    dg_list = []
    for i in range(len(dg)):
      # if dg['ICD9_CODE'][i][0] != 'V' and dg['ICD9_CODE'][i][0] != 'E':
      if dg['ICD9_CODE'][i] in dict_ICD9.keys():
        dg_list.append(dict_ICD9[dg['ICD9_CODE'][i]])
      else:
        dict_ICD9[dg['ICD9_CODE'][i]] = dict_cnt
        dg_list.append(dict_cnt)
        dict_cnt += 1
    diag.append(dg_list)

  times.append(time)
  diags.append(diag)
  labels.append(label)

In [145]:
len(diags)

0

In [130]:
test = []
test.append(diags[0:len(diags)//5])
test.append(labels[0:len(diags)//5])
test.append(times[0:len(diags)//5])

train = []
train.append(diags[len(diags)//5:])
train.append(labels[len(diags)//5:])
train.append(times[len(diags)//5:])

In [131]:
import random
subject_id_neg = list(admissions[~admissions.DIAGNOSIS.str.contains('HEART FAILURE', na=False)]['SUBJECT_ID'])
subject_id_neg = random.sample(subject_id_neg, len(subject_id_HF)*6)

In [132]:
import math
diags = []
labels = []
times = []
for s_id in subject_id_neg:
  adm = admissions[admissions.SUBJECT_ID == s_id]
  adm.sort_values(by=['ADMITTIME'], inplace = True)
  adm = adm.reset_index()
  if len(adm) <= 2:
    continue
  diag = []
  label = 0
  time = []
  for i in range(len(adm)-1):
    date_obj = datetime.strptime(adm['ADMITTIME'][i][2:], '%y-%m-%d %H:%M:%S')\
                - datetime.strptime(adm['ADMITTIME'][len(adm)-2][2:], '%y-%m-%d %H:%M:%S')
    time.append(0 - date_obj.days)

    dg = diagnoses[diagnoses.HADM_ID == adm['HADM_ID'][i]].reset_index()
    dg_list = []
    for i in range(len(dg)):
      if type(dg['ICD9_CODE'][i]) != float: # and dg['ICD9_CODE'][i][0] != 'V' and dg['ICD9_CODE'][i][0] != 'E':
        # dg_list.append(dg['ICD9_CODE'][i])
        if dg['ICD9_CODE'][i] in dict_ICD9.keys():
          dg_list.append(dict_ICD9[dg['ICD9_CODE'][i]])
        else:
          dict_ICD9[dg['ICD9_CODE'][i]] = dict_cnt
          dg_list.append(dict_cnt)
          dict_cnt += 1

    diag.append(dg_list)

  times.append(time)
  diags.append(diag)
  labels.append(label)

times = times[0: 3*(len(test[0])+len(train[0]))]
diags = diags[0: 3*(len(test[0])+len(train[0]))]
labels = labels[0: 3*(len(test[0])+len(train[0]))]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [133]:
test[0].extend(diags[0:len(diags)//5])
test[1].extend(labels[0:len(diags)//5])
test[2].extend(times[0:len(diags)//5])

train[0].extend(diags[len(diags)//5:])
train[1].extend(labels[len(diags)//5:])
train[2].extend(times[len(diags)//5:])

In [134]:
fw = open('KIDNEY_training.pickle','wb')
pickle.dump(train, fw)
fw.close()

fw = open('KIDNEY_testing.pickle','wb')
pickle.dump(train, fw)
fw.close()

fw = open('KIDNEY_code2idx.pickle','wb')
pickle.dump(dict_ICD9, fw)
fw.close()

In [135]:
len(train[0])

708